# 술 추천 알고리즘

도수 유사도는 `euclidean distance`로 구했고, 풍미 및 기타 정보의 유사도는 `cosine similarity`로 구했다.  
술의 고유 id를 받아 비슷한 술의 고유 id를 return한다.

### general_recommendation():
도수 유사도 50% 풍미(6가지) 유사도 50% 고려해 주종 상관 없이 비슷한 술을 찾아줌

### wine_recommendation():
도수 유사도 50% 기타(category, origin, producer, wine_grape, flavour) 50% 고려해 같은 대분류 내에서 비슷한 와인을 찾아줌

### the used variables of each alcohols
(현재는 wine recommendation만 구축됨! makgeoli, beer, vodka, soju, whisky, korean도 variable만 바꾸어주어 만들면 됨.)  

all: percent, flavour  
wine: category, percent, **origin**, **producer**, **wine_grape**, flavour  
makgeoli: category, percent, **producer**, flavour  
beer: category, percent, **origin**, **producer**, flavour  
vodka: **origin**, producer, flavour  
soju: category, percent, **producer**, flavour  
whisky: category, **whisky_category**, percent, **origin**, flavour  
korean: category, percent, flavour  

### the explanation of variables
id: 술의 고유 id  
class: 주종  
category: 중분류  
name: 이름  
percent: 도수  
origin: 원산지/생산지  
producer: 제조사/생산자  
wine_grape: 포도 품종 (와인만)  
whisky_category: 위스키 소분류 (위스키만)  
sweet: 달콤한  
light: 가벼운  
soft: 부드러운  
bitter: 쓴맛이 강한  
clean: 깔끔한  
smell: 향이 강한  

In [ ]:
# 여러 주종 내 추천
# 주종에 상관 없이 도수와 풍미만 고려해 비슷한 술을 추천해줌
# 항목: 도수, 풍미 (약 6개 항목)

import numpy as np
import pandas as pd 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance_matrix
from scipy.spatial.distance import squareform
from sklearn.preprocessing import MinMaxScaler

data = pd.read_csv("data/final_data.csv")

# 확인
data.head()

,id,class,category,name,percent,origin,producer,wine_grape,whisky_category,sweet,light,soft,bitter,clean,smell
0,1,wine,와인-레드,"도멘 드 벨렌, 부르고뉴 메종 디유",13.0,프랑스,도멘 드 벨렌,피노 누아,NaN,1,1,0,1,1,0
1,2,wine,와인-레드,몬테스 알파 블랙 라벨 피노 누아,14.0,칠레,몬테스,피노 누아,NaN,0,0,0,1,0,0
2,3,wine,와인-레드,"바타시올로, 바롤로",13.5,이탈리아,바타시올로,네비올로,NaN,0,0,0,1,0,1
3,4,wine,와인-레드,아라고니아 셀렉시온 에스페시알,14.5,스페인,보데가스 아라고네사스,가르나차,NaN,1,0,0,0,0,0
4,5,wine,와인-스파클링,"메디치 에르메테, 콘체르토",11.5,이탈리아,메디치 에르메테,람브루스코 살라미노,NaN,1,0,1,0,0,1


In [ ]:
# 도수 유사도 구하기
percent = data['percent']
dist_pair = []

# y축에 임의로 0을 부여한 거리 순서쌍 생성
for i in range(0,len(percent)):
    temp = []
    temp.append(data.loc[i]['percent'])
    temp.append(0)
    dist_pair.append(temp)

# get a distance matrix
df = pd.DataFrame(dist_pair, columns=['x', 'y'])
dist_matrix = distance_matrix(df.values, df.values)

# 정규화
min_max_scaler = MinMaxScaler()
regularised = min_max_scaler.fit_transform(dist_matrix)

# 1에서 빼줘서 더 가까운 것이 우선순위를 갖도록 변경하기
one_matrix = np.ones((654,654))

final_dist = one_matrix - regularised

# 확인
print(final_dist)

[[1.         0.97560976 0.98795181 ... 0.82352941 0.82352941 0.82352941]
 [0.97619048 1.         0.98795181 ... 0.85154062 0.85154062 0.85154062]
 [0.98809524 0.98780488 1.         ... 0.83753501 0.83753501 0.83753501]
 ...
 [0.85       0.87073171 0.86024096 ... 1.         1.         1.        ]
 [0.85       0.87073171 0.86024096 ... 1.         1.         1.        ]
 [0.85       0.87073171 0.86024096 ... 1.         1.         1.        ]]


In [ ]:
# flavour word list 만들기
flavour_list = [] # empty list

for i in range(0, len(data)):
    temp = ""
    
    if data.loc[i]['sweet'] == 1:
        temp = temp + "sweet "
    if data.loc[i]['light'] == 1:
        temp = temp + "light "
    if data.loc[i]['soft'] == 1:
        temp = temp + "soft "
    if data.loc[i]['bitter'] == 1:
        temp = temp + "bitter "
    if data.loc[i]['clean'] == 1:
        temp = temp + "clean "
    if data.loc[i]['smell'] == 1:
        temp = temp + "smell "
    
    flavour_list.append(temp)

# 해당 리스트 데이터 프레임에 추가
data["flavour"] = pd.DataFrame({"flavour":flavour_list})
flavour = data['flavour']

# flavour 코사인 유사도 구하기
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(flavour)

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# 확인
print(cosine_sim)

[[1.         0.5        0.35355339 ... 0.57735027 0.5        0.70710678]
 [0.5        1.         0.70710678 ... 0.57735027 1.         0.        ]
 [0.35355339 0.70710678 1.         ... 0.81649658 0.70710678 0.        ]
 ...
 [0.57735027 0.57735027 0.81649658 ... 1.         0.57735027 0.        ]
 [0.5        1.         0.70710678 ... 0.57735027 1.         0.        ]
 [0.70710678 0.         0.         ... 0.         0.         1.        ]]


In [ ]:
# 도수, flavour를 모두 고려한 similarity 구하기 (weight는 각각 0.5)
new_sim = 0.5 * cosine_sim + 0.5 * final_dist

print(new_sim)

[[1.         0.73780488 0.6707526  ... 0.70043984 0.66176471 0.7653181 ]
 [0.73809524 1.         0.84752929 ... 0.71444544 0.92577031 0.42577031]
 [0.67082431 0.84745583 1.         ... 0.8270158  0.7723209  0.41876751]
 ...
 [0.71367513 0.72404099 0.83836877 ... 1.         0.78867513 0.5       ]
 [0.675      0.93536585 0.78367387 ... 0.78867513 1.         0.5       ]
 [0.77855339 0.43536585 0.43012048 ... 0.5        0.5        1.        ]]


In [ ]:
# 여러 주종 내 추천
# 주종에 상관 없이 도수와 풍미만 고려해 비슷한 술을 추천해줌
# 항목: 도수, 풍미 (약 6개 항목)

# 고유 id를 넣으면 해당 술과 비슷한 Top 10의 id를 return
def general_recommendation(input_id, new_sim = new_sim):
    
    # 고유 id로 index 찾기 
    idx = data.index[data['id'] == input_id].tolist() # Int64Index 형식이라 list로 바꾸어줌
    
    # 해당 index의 유사도 리스트 sort in descending order
    score_series = pd.Series(new_sim[idx[0]]).sort_values(ascending = False)
    
    # 유사도 Top 10의 index 추출
    top_10_indexes = list(score_series.iloc[1:11].index)

    # 유사도 1인 항목이 하나 더 있어서 자기 자신이 포함되는 경우에는 자신을 뺀 Top 10의 index 재추출
    if top_10_indexes[0] == idx[0]:
        top_10_indexes = list(score_series.iloc[1:12].index)
        top_10_indexes.remove(idx[0])
    
    # 고유 id를 담기 위한 empty list 생성
    top_10_id = []
    
    # id list
    for i in top_10_indexes:
        id = data.loc[i]['id']
        top_10_id.append(id)
    
    return top_10_id

# test
print(general_recommendation(10))

[323, 90, 383, 92, 192, 557, 136, 349, 397, 360]


In [ ]:
# wine word list 만들기
wine_list = [] # empty list

# wine index list 추출
wine_idx = data.index[data['class'] == "wine"].tolist()

# wine: category, percent, origin, producer, wine_grape, flavour
for i in wine_idx:
    temp = ""
    temp = temp + data.loc[i]['category'] + " " + data.loc[i]['origin'] + " " + data.loc[i]['producer'].replace(" ", "") + " " + data.loc[i]['wine_grape'].replace(" ", "") + " "
    
    for flavour in data.loc[i]['flavour']:
        temp = temp + flavour

    wine_list.append(temp)

# wine 코사인 유사도 구하기
# instantiating and generating the count matrix
count = CountVectorizer()
wine_matrix = count.fit_transform(wine_list)

# generating the cosine similarity matrix
wine_sim = cosine_similarity(wine_matrix, wine_matrix)

# wine 도수 유사도 구하기
wine_pair = []

# y축을 임의로 0을 부여한 거리 순서쌍 생성
for i in wine_idx:
    temp = []
    temp.append(data.loc[i]['percent'])
    temp.append(0)
    wine_pair.append(temp)

# get a distance matrix
wine_df = pd.DataFrame(wine_pair, columns=['x', 'y'])
wine_matrix = distance_matrix(wine_df.values, wine_df.values)

# 정규화
min_max_scaler = MinMaxScaler()
wine_regularised = min_max_scaler.fit_transform(wine_matrix)

# 1에서 빼줘서 더 가까운 것이 우선순위를 갖도록 변경하기
wine_one_matrix = np.ones((len(wine_idx),len(wine_idx)))

wine_final_dist = wine_one_matrix - wine_regularised

# 도수 유사도, 타 정보 유사도 각각 0.5씩 weight 부여 후 새로운 matrix 생성
wine_new_sim = 0.5 * wine_sim + 0.5 * wine_final_dist

# 확인
print(wine_new_sim)

[[1.         0.71660997 0.65957047 ... 0.65957047 0.55027799 0.63608276]
 [0.70966553 1.         0.70204326 ... 0.62489159 0.43589744 0.60416667]
 [0.65773224 0.70367725 1.         ... 0.64285714 0.38484398 0.62305335]
 ...
 [0.65773224 0.62652557 0.64285714 ... 1.         0.38484398 0.54590167]
 [0.57912415 0.5        0.43009285 ... 0.43009285 1.         0.45833333]
 [0.63608276 0.61111111 0.62489159 ... 0.54773991 0.42948718 1.        ]]


In [ ]:
# 해당 id의 술이 wine인지 체크
def is_wine(input_id):
    temp_idx = data.index[data['id'] == input_id].tolist() # Int64Index 형식이라 list로 바꾸어줌
    result = data.loc[temp_idx[0]]['class'] == "wine"
    return result

# 와인의 고유 id를 넣으면 해당 와인과 비슷한 Top 10 와인의 id를 return
def wine_recommendation(input_id, wine_new_sim = wine_new_sim):
    
    # 고유 id로 index 찾기 
    idx = data.index[data['id'] == input_id].tolist() # Int64Index 형식이라 list로 바꾸어줌
    
    # wine_idx list 내에서 몇번째 와인인지 구하기
    w_idx = wine_idx.index(idx[0])
    
    # 해당 index의 유사도 리스트 sort in descending order
    score_series = pd.Series(wine_new_sim[w_idx]).sort_values(ascending = False)
    
    # 유사도 Top 10의 index 추출
    wine_top_10_indexes = list(score_series.iloc[1:11].index)

    # 유사도 1인 항목이 하나 더 있어서 자기 자신이 포함되는 경우에는 자신을 뺀 Top 10의 index 재추출
    if wine_top_10_indexes[0] == w_idx:
        wine_top_10_indexes = list(score_series.iloc[1:12].index)
        wine_top_10_indexes.remove(w_idx)
    
    # 고유 id를 담기 위한 empty list 생성
    wine_top_10_id = []
    
    # id list
    for i in wine_top_10_indexes:
        index = wine_idx[i] # wine list에서 몇번째인지가 아니라 전체 술 list에서 몇번째인지 구함
        id = data.loc[index]['id']
        wine_top_10_id.append(id)
    
    return wine_top_10_id

In [ ]:
# test
# 테스트로 임의의 아이디 넣음
test_num = 2

print(general_recommendation(test_num))

print(is_wine(test_num))

print(wine_recommendation(test_num))

[78, 54, 300, 308, 293, 512, 79, 547, 620, 638]
True
[19, 58, 43, 9, 1, 71, 3, 59, 6, 68]
